## Part 2A: Discrete shape analysis

In this notebook, we will learn the basics of shape analysis on landmarked data.

In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm

### 0. Data loading

**0.1** Run the lines below to load point-based object models of objects in the C. elegans dataset.

*The C. elegans dataset is presented and prepared in notebook 1 - Data preparation.*

In [ ]:
dataset='data/C. elegans/point_models.npy'
point_models=np.load(dataset)

In [ ]:
# The number of points in our object model is an important parameter
N=point_models.shape[1]
print("N="+str(N))

**0.2** Run the lines below to visualize a set of a few randomly selected models from the collection.

In [ ]:
number=10
inds=np.random.choice(len(point_models)-1, size=number, replace=False)

fig, ax = plt.subplots(1,number, figsize=(number,1))
for i in range(number):
    point_model=np.array(point_models[inds[i]])
    ax[i].scatter(point_model[:,0],point_model[:,1],s=1)

### 1. Procrustes alignment

**1.1** Center and scale the data such that they are all located around (0,0) and are of unit norm. Object models normalized in this way are referred to as preshapes.

*Hint: use ``np.linalg.norm`` to normalize*

In [ ]:
# Desired output: preshapes array of the same dimensions as point_model
# Add your code here!
preshapes=

**1.2** Write an objective function to align one object model onto another one, assuming that the two models are in correspondence†. This problem can be formulated as finding the rotation angle that minimizes the root mean square error (RMSE) between the two point sets.

*Hint 1: assuming the distance between two models is ``L``, the RMSE is given as ``np.sqrt(np.mean(L**2))``.*

*Hint 2: rotating a point set can be done by multiplying with a 2D rotation matrix (https://en.wikipedia.org/wiki/Rotation_matrix).*

† Bonus question: do you understand what this assumption means? Which step must be performed when building the object models to ensure that it holds?

In [ ]:
import scipy.optimize

In [ ]:
# Picks two random models - reload if you are not happy with the selection :)
random=np.random.choice(len(point_models)-1, size=2, replace=False)

sample_preshape_1=preshapes[random[0]]
sample_preshape_2=preshapes[random[1]]

plt.scatter(sample_preshape_1[:,0],sample_preshape_1[:,1])
plt.scatter(sample_preshape_2[:,0],sample_preshape_2[:,1])
plt.show()

In [ ]:
# Desired output: objectiveFunctionAlignment function returning the RMSE between a reference model m1 and 
# another model m2 rotated by an angle theta
# Add your code here!
def objectiveFunctionAlignment(theta, m1, m2):

In [ ]:
# Optimizes the objective function
res=scipy.optimize.minimize(objectiveFunctionAlignment,0,args=(sample_preshape_1,sample_preshape_2),method="L-BFGS-B")
theta0=res.x[0]
print(theta0)

In [ ]:
# Visualizes the alignment
plt.scatter(sample_preshape_1[:,0],sample_preshape_1[:,1])

R=np.array([[np.cos(theta0),-np.sin(theta0)],[np.sin(theta0),np.cos(theta0)]])
aligned=(R @ sample_preshape_2.T).T

plt.scatter(aligned[:,0],aligned[:,1])
plt.show()

**1.3** Object models in 2D can be seen as vectors of complex numbers associated with an appropriate Hermitian inner product. The lines below transforms your two objects into complex vectors. Run the lines below to convince yourself that the "vector of complex numbers" representation is equivalent to the "vector of points" one.

In [ ]:
complex_sample_preshape_1=sample_preshape_1[:,0]+1j*sample_preshape_1[:,1]
complex_sample_preshape_2=sample_preshape_2[:,0]+1j*sample_preshape_2[:,1]

plt.scatter(complex_sample_preshape_1.real,complex_sample_preshape_1.imag)
plt.scatter(complex_sample_preshape_2.real,complex_sample_preshape_2.imag)
plt.show()

**1.4** The solution of the optimal alignment problem 1.2 can actually be expressed in closed-form when relying on complex representations, bypassing the need for an optimization. Run the lines below to verify that you obtain the same solution as in 1.2 (and be amazed by the beauty of maths :)).

In [ ]:
theta0_complex=np.angle(complex_sample_preshape_2.conj().T @ complex_sample_preshape_1)
print(theta0_complex)

In [ ]:
# Visualizes the alignment 
plt.scatter(complex_sample_preshape_1.real,complex_sample_preshape_1.imag)

aligned_complex=np.exp(1j*theta0_complex)*complex_sample_preshape_2

plt.scatter(aligned_complex.real,aligned_complex.imag)
plt.show()

*In case you are wondering what the hell happened in the lines above, here are more details (if you don't care, you can safely skip to 2 below).*

To align the two objects, we must retreive the angle between them. If what we were dealing with was 2D vectors, we would get that angle immediately through the scalar product (https://en.wikipedia.org/wiki/Dot_product). An equivalent of the scalar product can be defined for complex numbers (usually referred to as Hermitian inner product, https://en.wikipedia.org/wiki/Inner_product_space), so we could equivalently retreive that angle this way as demonstrated in the code below.

Now it happens that we are not working with 2D vectors (or complex numbers), but with Nx2 matrices (or complex vectors)! Fortunately, the Hermitian inner product adapt straighforwardly to complex vectors, leading to the expression you see above.

In [ ]:
# Define two arbitrary 2D vectors v1 and v2 and normalize them
v1=[10,7]
v1/=np.linalg.norm(v1)

v2=[5,5]
v2/=np.linalg.norm(v2)

# Compute the angle between v1 and v2 using the definition of the scalar product
angle_real=np.arccos(np.dot(v1,v2))

# Turn v1 and v2 into z1 and z2, their complex representation
z1=v1[0]+1j*v1[1]
z2=v2[0]+1j*v2[1]

# Compute the angle between z1 and z2 using the definition of the Hermitian (complex) inner product
angle_complex=np.angle(z1.conj()*z2)

# Display the results
print("Angle computed with the real inner product: "+str(angle_real))
print("Angle computed with the complex inner product: "+str(angle_complex))

### 2. Kendall shape space

**2.1** To build the shape space, we will rely on the complex representation introduced in 1.3. Convert your whole collection of preshapes (obtained in 1.1) into complex numbers. 

*Important note: Make sure that complex_preshapes is a numpy array!*

In [ ]:
# Desired output: complex_preshapes array of the same length as preshapes
# Add your code here!
complex_preshapes=

**2.2** The function below computes the Fréchet mean of a dataset relying on complex number representations. Use it to extract the Fréchet mean of your dataset and visualize it.

In [ ]:
def meanFrechet(input_complex_preshapes): # preshapes
    '''Input: dataset of complex preshapes.
    Output: Fréchet mean (w.r.t. the Procrustes distance).'''
    SQ = input_complex_preshapes.T @ input_complex_preshapes.conj()
    D,V = np.linalg.eig(SQ)
    ds = np.real(D)
    ind_max = np.argmax(ds)
    ds_max = np.max(ds)
    m = V[:,ind_max]
    
    centered_m=m-np.mean(m)
    m_norm=np.sqrt(centered_m.conj().T @ centered_m).real
    return m/m_norm

In [ ]:
# Desired output: mean_shape_Frechet array
# Add your code here!
mean_shape_Frechet=

*In case you are wondering what the hell happened in the lines above, here are more details (if you don't care, you can safely skip to 2.3 below).*

It is not straightforward to see how the minimization problem one needs to solve to obtain the Fréchet mean transforms into an eigenvalue decomposition as implemented in the function meanFrechet above. Those interested in more details and a proof are invited to look at Result 8.2 (Section 8.3) of the Dryden & Mardia textbook (ISBN 978-0471958161).

**2.3** Using the alignment procedure introduced in 1.4, align each element of the dataset onto the Fréchet mean (computed in 2.2).

In [ ]:
# Desired output: complex_aligned_shapes array of the same dimensions as complex_preshapes
# Add your code here!
complex_aligned_shapes=

**2.4** Now that all shapes are aligned, compute the (point-by-point) mean shape of the dataset and visualize it. Does it differ from what you got in 2.2?

In [ ]:
# Desired output: mean_shape array and its scatter plot
# Add your code here!
mean_shape=

**2.5** The code below and retreives the mean shape of the dataset relying on general Procrustes alignment. Run it a few times: what do you observe? Does the mean shape obtained in this way differ from what you got in 2.4? 

*As a reminder, general Procrustes alignment  the following iterative procedure:*
1. *Randomly select one object from the collection*
2. *Align every other objects to it (ordinary Procrustes alignment)*
3. *Compute the average of all aligned data points and scale it to unit norm*
4. *Align all objects to that new average*
5. *Compute the average of all aligned data (not considering the average obtained at step 3 as this is not part of the data!)*
6. *Repeat steps 4 and 5 until convergence*

In [ ]:
from copy import deepcopy

# 1. Randomly select one object from the collection
random=np.random.randint(len(complex_preshapes)-1)
complex_reference=complex_preshapes[random]

# 2. Align every other objects to it (ordinary Procrustes alignment)
aligned_complex=np.zeros(complex_preshapes.shape, dtype=complex)
for i in range(len(complex_preshapes)):
    aligned_complex[i]=(complex_preshapes[i].conj().T @ complex_reference)*complex_preshapes[i]
    
# 3. Compute the average of all aligned data points and scale it to unit norm
mean_shape_2=np.mean(aligned_complex,0)
mean_shape_2/=np.linalg.norm(mean_shape_2)

epsilon = 1e-3
delta = np.Inf
while np.abs(delta) > epsilon:
    # 4. Align all objects to that new average
    aligned_complex=np.zeros(complex_preshapes.shape, dtype=complex)
    for i in range(len(complex_preshapes)):
        aligned_complex[i]=(complex_preshapes[i].conj().T @ mean_shape_2)*complex_preshapes[i]

    # 5. Compute the average of all aligned data (not considering the average obtained at step 3 as this is not part of the data!)
    new_mean_shape_2=np.mean(aligned_complex,0)
    new_mean_shape_2/=np.linalg.norm(new_mean_shape_2)

    # 6. Repeat steps 4 and 5 until convergence
    delta=np.linalg.norm(new_mean_shape_2 - mean_shape_2)
    mean_shape_2=deepcopy(new_mean_shape_2)

# Visualize the result
plt.scatter(mean_shape_2.real,mean_shape_2.imag)
plt.show()

**2.6** Relying on the function ``geodesicPath`` below, compute the geodesic path between a randomly picked shape in the dataset and the Fréchet mean, and visualize elements along the path.

In [ ]:
def geodesicDistance(z,w): 
    '''Geodesic distance between [z] and [w], but computed on preshapes.'''
    aux=np.abs(z.conj().T @ w)
    if aux>1.0: 
        aux=1.0
    return np.arccos(aux)

def geodesicPath(z,w,numSteps): 
    '''Returns elements regularly spaced along the geodesic curve joining z to w (preshapes).'''
    ro = geodesicDistance(z,w)
    steps = np.arange(numSteps+1)/numSteps

    ta = np.angle(z.conj().T @ w)
    path = 1/np.sin(ro)*(np.sin((1-steps[:,None])*ro)*np.exp(1j*ta)*z + np.sin(steps[:,None]*ro)*w)
    return path

In [ ]:
# Picks one random complex preshape and visualize it with the Fréchet mean - reload if you are not happy with the selection :)
random=np.random.choice(len(complex_preshapes)-1, size=1, replace=False)[0]
sample_preshape=complex_preshapes[random]

plt.scatter(mean_shape_Frechet.real,mean_shape_Frechet.imag)
plt.scatter(sample_preshape.real,sample_preshape].imag)
plt.show()

In [ ]:
# Desired output: path array containing steps number of object models corresponding to shapes on the geodesic path
# between mean_shape_Frechet and sample_preshape, and scatter plots visualizing them
# Add your code here!
steps=
path=

**2.7** Compare the length of the above geodesics with the Procrustes distance (the distance you used for the alignment in 1.2). What do you observe? Can you relate that to what you know about the nature of the shape space?

In [ ]:
# Desired output: geodesic distance between sample_preshape and mean_shape_Frechet, and Euclidean distance between
# complex_aligned_shapes[random] and mean_shape_Frechet.
# Add your code here!
geo_distance=
print("Geodesic distance: "+str(geo_distance))

procrustes_dist=
print("Procrustes distance: "+str(procrustes_dist))

**2.8** Compare the Procrustes distance (the distance you used for the alignment in 1.2) with the chord distance on the shape space defined below. What do you observe?

In [ ]:
def chordDistance(z,w): # defined on preshapes
    '''Chord distance between [z] and [w].'''
    return np.sqrt(2.0 - 2.0*np.abs(z.conj().T @ w))

In [ ]:
# Desired output: chord distance between sample_preshape and mean_shape_Frechet, and Euclidean distance between
# complex_aligned_shapes[random] and mean_shape_Frechet.
# Add your code here!
chord_distance=
print("Chord distance: "+str(chord_distance))
print("Procrustes distance: "+str(procrustes_dist))

### 3. Shape space PCA on the whole dataset

A classical trick to perform local linear operations on a Riemannian manifold is to rely on local tangent spaces. The way into and out of the tangent space are the logarithm and exponential maps, respectively. Their expression for the shape space of point-based models we are currently working with are provided below.

In [ ]:
def logarithmMap(z,w): 
    '''Computes a preshape pertaining to the equivalence class log_[z]([w]),
    where log is relative the shape space Sigma'''
    ta = np.angle(z.conj().T @ w)
    w_r = np.exp(-1j*ta)*w
    ro = geodesicDistance(z,w_r)
    return ro/np.sin(ro)*(w_r - np.cos(ro)*z)

def exponentialMap(z,v): 
    '''Computes the exponential of the tangent vector v in C^n at a preshape z'''
    t = np.sqrt(v.conj().T @ v).real
    if t < 1e-16 :
        return z
    return np.cos(t)*z + v*np.sin(t)/t

**3.1** The lines below perform PCA in the tangent plane around the Fréchet mean using the logarithmic and exponential maps appropriately and displays the explained variance. What do you notice?

In [ ]:
from sklearn.decomposition import PCA

In [ ]:
tangent_vectors=np.zeros((len(complex_preshapes),2*N))
for i in range(len(complex_preshapes)):
    tangent_vector=logarithmMap(mean_shape_Frechet, complex_preshapes[i])
    tangent_vectors[i,:N]=tangent_vector.real
    tangent_vectors[i,N:]=tangent_vector.imag

tangent_plane_pca = PCA()
transformed_vectors = tangent_plane_pca.fit_transform(tangent_vectors)

plt.plot(np.linspace(1,tangent_plane_pca.n_components_,tangent_plane_pca.n_components_),100*tangent_plane_pca.explained_variance_/np.sum(tangent_plane_pca.explained_variance_))
plt.xlabel("Principal component")
plt.ylabel("Variance explained")
plt.show()

**3.2** The lines below retreive the four first modes of shape variation and visualize them around the mean. What does it tell you about the shapes present in the data?

In [ ]:
l=np.sqrt(tangent_plane_pca.explained_variance_)
K=4

modes_shape=np.zeros((K,2,N),dtype=np.complex)
for i in range(K) :
    vector=np.zeros(tangent_plane_pca.n_components_)
    vector[i]=1
    mode=l[i]*tangent_plane_pca.inverse_transform(vector)
    complex_mode=mode[:N]+1j*mode[N:]
    modes_shape[i][0]=exponentialMap(mean_shape_Frechet, complex_mode)
    modes_shape[i][1]=exponentialMap(mean_shape_Frechet, -complex_mode)
    
fig, ax = plt.subplots(1, K, figsize=(5*K,5))
for i in range(K):
    ax[i].scatter(mean_shape_Frechet.real,mean_shape_Frechet.imag)
    ax[i].scatter(modes_shape[i][0].real, modes_shape[i][0].imag)
    ax[i].scatter(modes_shape[i][1].real, modes_shape[i][1].imag)
    
    percent_variance=np.round(100*l[i]**2/np.sum(tangent_plane_pca.explained_variance_))
    ax[i].set_title("Mode "+str(i+1)+", "+str(percent_variance)+"% of variance.")

**3.3** The lines below plot the PCA-transformed data around the Fréchet mean. Each point corresponds to an object. If z_m is the Fréchet mean and l_1, l_2 are the two first eigenvalues of the PCA, the red point corresponds to z_m, the blue points to z_m +/- l_1, and the cyan point to z_m +/- l_2.

What does this plot tell you about the dataset?

In [ ]:
plt.scatter(transformed_vectors[:,0], transformed_vectors[:,1], c="g")
plt.scatter([0],[0], c="r")
plt.scatter([l[0],-l[0]], [0.,0.],c="blue")
plt.scatter([0.,0.],[l[1],-l[1]],c="cyan")
plt.axis("equal")
plt.title("PCA plot around the Fréchet mean")
plt.show()

**[BONUS] 3.4** The lines below perform PCA as above but without using the logarithm and exponential maps. What do you see? Can you relate that to what you observed in 2.7-2.8?

In [ ]:
# Performs PCA assuming the shape space is linear
flattened_aligned_shapes=np.zeros((len(complex_aligned_shapes),2*N))
for i in range(len(complex_aligned_shapes)):
    flattened_aligned_shapes[i,:N]=complex_aligned_shapes[i].real
    flattened_aligned_shapes[i,N:]=complex_aligned_shapes[i].imag
    
flat_pca = PCA()
flat_transformed_data = flat_pca.fit_transform(flattened_aligned_shapes)

plt.plot(np.linspace(1,flat_pca.n_components_,flat_pca.n_components_),100*flat_pca.explained_variance_/np.sum(flat_pca.explained_variance_))
plt.xlabel("Number of principal components")
plt.ylabel("Variance explained")
plt.show()

In [ ]:
# Visualizes the first K modes of variation
flat_l=np.sqrt(flat_pca.explained_variance_)
K=4

flat_modes_shape=np.zeros((K,N,2))
for i in range(K) :
    vector=np.zeros(flat_pca.n_components_)
    vector[i]=1
    flat_mode=0.25*flat_l[i]*flat_pca.inverse_transform(vector)
    flat_modes_shape[i][:,0]=flat_mode[:N]
    flat_modes_shape[i][:,1]=flat_mode[N:]
    
fig, ax = plt.subplots(1, K, figsize=(5*K,5))
for i in range(K):
    ax[i].scatter(mean_shape_Frechet.real,mean_shape_Frechet.imag)
    ax[i].scatter(flat_modes_shape[i][:,0], flat_modes_shape[i][:,1])
    ax[i].scatter(-flat_modes_shape[i][:,0], -flat_modes_shape[i][:,1])
    
    percent_variance=np.round(100*flat_l[i]**2/np.sum(flat_pca.explained_variance_))
    ax[i].set_title("Mode "+str(i+1)+", "+str(percent_variance)+"% of variance.")

### 4. Shape space PCA on classes

We actually know that the shape distribution is bimodal, because there are dead and alive worms. To have a clearer picture of shape variability, we should thus carry out shape PCA on each class individually.

**4.1** Run the lines below to split the dataset into a collection of alive and a collection of dead C.elegans samples.  

In [ ]:
label_data='data/C. elegans/labels.npy'
labels=np.load(label_data)

In [ ]:
complex_preshapes_dead=complex_preshapes[np.where(labels==0)]
complex_preshapes_live=complex_preshapes[np.where(labels==1)]

**4.2** Run the lines below to visualize a set of a few randomly selected models from each collection.

In [ ]:
# Dead C. elegans
number=5
inds=np.random.choice(len(complex_preshapes_dead)-1, size=number, replace=False)

fig, ax = plt.subplots(1,number, figsize=(number,1))
for i in range(number):
    sample=complex_preshapes_dead[inds[i]]
    ax[i].scatter(sample.real,sample.imag,s=1)
    ax[i].set_title(inds[i])

In [ ]:
# Live C. elegans
number=5
inds=np.random.choice(len(complex_preshapes_live)-1, size=number, replace=False)

fig, ax = plt.subplots(1,number, figsize=(number,1))
for i in range(number):
    sample=complex_preshapes_live[inds[i]]
    ax[i].scatter(sample.real,sample.imag,s=1)

**4.3** Compute and visualize the Fréchet mean for each individual class, reusing the `meanFrechet` function from 2.2. How do they differ from what you got in 2.2?

In [ ]:
# Desired output: mean_shape_Frechet_dead and mean_shape_Frechet_live arrays, and scatter plots to visualize them
# Add your code here!
mean_shape_Frechet_dead=
mean_shape_Frechet_live=

**4.4** For each class, perform PCA in the tangent plane around the Fréchet mean by adapting the code from 3.1.

In [ ]:
# Desired output: tangent plane PCA as in 3.1 for complex_preshapes_dead around mean_shape_Frechet_dead (tangent_plane_pca_dead) 
# and complex_preshapes_live around mean_shape_Frechet_live (tangent_plane_pca_live)
# Add your code here!
tangent_plane_pca_dead =
tangent_plane_pca_live =

**4.5** For each class, retreive the two first modes of shape variation and visualize them around the mean by adapting the code from 3.2. How do they differ from what you got in 3.2?

In [ ]:
# Desired output: mode visualisation as in 3.2 for tangent_plane_pca_dead (modes_shape_dead) and 
# tangent_plane_pca_alive (modes_shape_alive)
# Add your code here!
modes_shape_dead=
modes_shape_live=

**4.6** The lines below plot the PCA-transformed data around the Fréchet mean for each class (see 3.3 for a detailed explanations of what the different coloured points correspond to). How does it compare to 3.3?

In [ ]:
fig, ax = plt.subplots(1,2)

ax[0].scatter(transformed_vectors_dead[:,0], transformed_vectors_dead[:,1], c="g")
ax[0].scatter([0],[0], c="r")
ax[0].scatter([l_dead[0],-l_dead[0]], [0.,0.],c="blue")
ax[0].scatter([0.,0.],[l_dead[1],-l_dead[1]],c="cyan")
ax[0].axis("equal")
ax[0].set_title("Dead")

ax[1].scatter(transformed_vectors_live[:,0], transformed_vectors_live[:,1], c="g")
ax[1].scatter([0],[0], c="r")
ax[1].scatter([l_live[0],-l_live[0]], [0.,0.],c="blue")
ax[1].scatter([0.,0.],[l_live[1],-l_live[1]],c="cyan")
ax[1].axis("equal")
ax[1].set_title("Live")

plt.suptitle("PCA plot around the Fréchet mean")
plt.show()

### 5. Statistical shape modelling

**5.1** The following lines generate synthetic live C. elegans shapes relying on a simple statistical model (multivariate Gaussian) built from the covariance matrix of the dataset. Do you understand how the model is constructed? How good do you think this model is and why?

In [ ]:
import scipy.stats

In [ ]:
count=3
num_synthetic_data=5

In [ ]:
# Live C. elegans
synthetic_data_live=np.zeros((num_synthetic_data,N),dtype=np.complex)
for i in range(num_synthetic_data):
    rand=[]
    for k in range(tangent_plane_pca_live.n_components_):
        var=tangent_plane_pca_live.explained_variance_[k]*(len(tangent_vectors_live)-1)
        if var<1e-6:
            sigma=0.0
        else:
            sigma=np.sqrt(var)
        rand.append(scipy.stats.norm.rvs(loc=0, scale=sigma, size=1))
    rand=np.array(rand)

    v=np.zeros((tangent_plane_pca_live.n_components_))
    for k in range(count):
        zz=rand.T @ tangent_plane_pca_live.components_.T[:,k]
        v+=(zz*tangent_plane_pca_live.components_.T[:,k])
    v+=tangent_plane_pca_live.mean_
    
    complex_v=v[:N]+1j*v[N:]
    synthetic_data_live[i]=exponentialMap(mean_shape_Frechet_live, complex_v) 
    
fig, ax = plt.subplots(1, num_synthetic_data, figsize=(num_synthetic_data,1))
for i in range(num_synthetic_data):
    ax[i].scatter(synthetic_data_live[i].real, synthetic_data_live[i].imag, s=5)
    ax[i].axis("off")